In [1]:
import re
from gensim import corpora
import gensim
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem import WordNetLemmatizer
import tqdm
import zlib
import base64
import os
import nltk
import pandas as pd
from summarizer import Summarizer
import string
from sklearn.feature_extraction.text import TfidfVectorizer

from sentence_transformers import SentenceTransformer
import numpy as np


from nltk import bigrams

In [2]:
merged_df = pd.read_csv('merged_df.csv')
merged_df.head()

,Text,DATE,rate,rateChange
0,meeting federal open market committee held off...,2000-02-02,5.75,0.25
1,meeting federal open market committee held off...,2000-03-21,6.00,0.50
2,meeting federal open market committee held off...,2000-05-16,6.50,0.00
3,meeting federal open market committee held off...,2000-06-28,6.50,0.00
4,meeting federal open market committee held off...,2000-08-22,6.50,0.00


In [3]:
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import BertTokenizer, BertForSequenceClassification, AdamW, get_linear_schedule_with_warmup
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, f1_score
from tqdm import tqdm
import numpy as np
import pandas as pd


In [7]:
merged_df["Label"] = merged_df["rateChange"].apply(lambda x: 1 if x > 0 else 2 if x < 0 else 0)

# Split data into training and test sets
train_texts, test_texts, train_labels, test_labels = train_test_split(
    merged_df['Text'].values, merged_df['Label'].values, test_size=0.5, random_state=42
)

# Load BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')


In [ ]:

# Tokenize the text
train_encodings = tokenizer(list(train_texts), truncation=True, padding=True, max_length=1024)
test_encodings = tokenizer(list(test_texts), truncation=True, padding=True, max_length=128)

# Convert to PyTorch Dataset
class FedDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

train_dataset = FedDataset(train_encodings, train_labels)
test_dataset = FedDataset(test_encodings, test_labels)


In [11]:
all_lens = []
for i in range(len(merged_df["Text"])):
    all_lens.append(len(merged_df["Text"][i]))

avg_len = sum(all_lens) / len(all_lens)
print(f"Average length of text: {avg_len}")
print(f"Max length of text: {max(all_lens)}")

Average length of text: 11334.471204188481
Max length of text: 16561


In [8]:

# Load pre-trained BERT model for sequence classification
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=3)  # 3 classes: -1, 0, 1

# Use GPU if available
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)


# Define training parameters
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)
optimizer = AdamW(model.parameters(), lr=2e-5)
num_epochs = 3
total_steps = len(train_loader) * num_epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Pl

In [6]:

# Training loop
model.train()
for epoch in range(num_epochs):
    total_loss = 0
    for batch in tqdm(train_loader, desc=f"Training Epoch {epoch+1}/{num_epochs}"):
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to
        (device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        scheduler.step()
        
        total_loss += loss.item()
    
    avg_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch+1}/{num_epochs} - Average Loss: {avg_loss:.4f}")

Training Epoch 1/3:   0%|          | 0/12 [00:03<?, ?it/s]


ValueError: Target size (torch.Size([8])) must be the same as input size (torch.Size([8, 3]))

In [ ]:
# Evaluation
model.eval()
predictions = []
true_labels = []

with torch.no_grad():
    for batch in test_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        preds = torch.argmax(logits, dim=1).cpu().numpy()
        
        predictions.extend(preds)
        true_labels.extend(labels.cpu().numpy())

# Calculate F1 score
f1 = f1_score(true_labels, predictions, average='weighted')
print(f'F1 Score: {f1:.4f}')
print(classification_report(true_labels, predictions, target_names=['Decrease', 'No Change', 'Increase']))